![Semillero Astrofisica](../logo.PNG)

---
# Ecuaciones Diferenciales Ordinarias: Introducción 


Carlos Andrés del Valle (cdelv@unal.edu.co)

---

Vamos a resolver la ecuación

\begin{equation}
    y^{\prime\prime}(t)-2y^\prime(t)+10y(t)=0
\end{equation}

Con las condiciones iniciales y de frontera
\begin{equation}
    y(t)=-2; \qquad y^\prime(t)=1
\end{equation}

La solución analítica es 
\begin{equation}
    y(t)=e^t\left(\sin{(3t)}-2\cos{(3t)}\right)
\end{equation}

In [1]:
import deepxde as dde
import numpy as np

#Usar doble precisión si la GPU lo soporta. Float32 por defecto.
dde.config.real.set_float64()

#para cambiar el backend
#dde.backend.set_default_backend('tensorflow')
#!export DDE_BACKEND=tensorflow

def func(t):
    return np.exp(t)*(np.sin(3*t)-2*np.cos(3*t))

Using backend: tensorflow

2022-10-21 15:30:38.640544: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-21 15:30:38.643666: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-21 15:30:38.643677: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


ModuleNotFoundError: No module named 'tensorflow_probability'

## 1. Definir la Ecuación a Resolver

In [ ]:
# Ecuación diferencial a resolver
# t es el vector de coordenadas que entra a la red, en este caso el tiempo
# y es el output de la red neuronal 
def ode(t, y):
    dy_dt = dde.grad.jacobian(y, t)
    d2y_dt2 = dde.grad.hessian(y, t)
    return d2y_dt2 - 2*dy_dt + 10*y

## 2. Definir Dominio de la Ecuación

Como tenemos solo dependencia temporal, podemos aprovechar el dominio TimeDomain

~~~python
class deepxde.geometry.timedomain.TimeDomain(t0, t1)
~~~

Esta clase de dominio crea el booleano para definir las coondiciones iniciales

~~~pyhton
on_initial(t)
~~~

In [ ]:
geom = dde.geometry.TimeDomain(0, 3)

## 3. Definir Condiciones Iniciales y de Frontera

Como estamos resolviendo una ODE, entonces solo tenemos dos condiciones iniciales: $y(0)=-1$;  $y^\prime(0)=0$. Para la condición inicial en la variable:

~~~python
class deepxde.icbc.initial_conditions.IC(geom, 
                                         func, 
                                         on_initial, 
                                         component=0)
~~~

Initial conditions: y([x, t0]) = func([x, t0]). Para la condición sobre la derivada:

~~~python
class deepxde.icbc.boundary_conditions.OperatorBC(geom, 
                                                  func, 
                                                  on_boundary)
~~~

General operator boundary conditions: func(inputs, outputs, X) = 0. 

- **func:** A function takes arguments (inputs, outputs, X) and outputs a tensor of size N x 1, where N is the length of inputs. inputs and outputs are the network input and output tensors, respectively; X are the NumPy array of the inputs.

Ver detalles en https://deepxde.readthedocs.io/en/latest/modules/deepxde.icbc.html?highlight=boundary_conditions.OperatorBC#deepxde.icbc.boundary_conditions.OperatorBC

In [ ]:
def in_bdr(X, on_initial):
    return on_initial and np.isclose(X[0], 0)

def IC_1(inputs):
    return -2

def IC_2(inputs, outputs, X):
    return dde.grad.jacobian(outputs, inputs) - 1 #=0 #se define igual que la ecuación

Ic1 = dde.icbc.IC(        geom, IC_1, in_bdr)
Ic2 = dde.icbc.OperatorBC(geom, IC_2, in_bdr)

## 4. Crear Datos de Entrenamiento

vamos a crear los datos de entrenamiento

~~~python
class deepxde.data.pde.TimePDE(geometryxtime, 
                               pde, 
                               ic_bcs, 
                               num_domain=0, 
                               num_boundary=0, 
                               num_initial=0, 
                               train_distribution='Hammersley', 
                               anchors=None, 
                               exclusions=None, 
                               solution=None, 
                               num_test=None, 
                               auxiliary_var_function=None
~~~

Detalles en https://deepxde.readthedocs.io/en/latest/modules/deepxde.data.html?highlight=data.TimePDE#deepxde.data.pde.TimePDE. 

In [ ]:
data = dde.data.TimePDE(geom, ode, [Ic1, Ic2], 62, 4, solution=func, num_test=100)

## 5. Crear la Red Neuronal

Vamos a utilizar una **Fully-connected neural network**. Existe la **Parallel fully-connected neural network** que usa una subred para cada salida. 

~~~python
class deepxde.nn.tensorflow.fnn.FNN(layer_sizes, activation, 
                                    kernel_initializer, 
                                    regularization=None, 
                                    dropout_rate=0)
~~~
Existen muchas funciones de activación, inicializadores y reguladores. Se pueden usar los que vienen hechos por defecto en el back-end que estamos utilizando, en este caso **TensorFlow**. Dejo donde revisar las diferentes opciones

- **Funciones de Activación:** https://www.tensorflow.org/api_docs/python/tf/keras/activations
- **Inicializadores:** https://www.tensorflow.org/api_docs/python/tf/keras/initializers
- **Reguladores:** https://www.tensorflow.org/api_docs/python/tf/keras/regularizers

In [ ]:
layer_size = [1] + [30]*3 + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

## 6. Compilar y Entrenar el Modelo

DeepXDE tiene una función que se encarga de compilar el modelo con el optimizador, learnign rate, métricas y estrategias de entrenamiento que queramos. Nuevamente, hay muchas opciones disponibles. 

~~~python
class deepxde.model.Model.compile(optimizer, 
                                  lr=None, 
                                  loss='MSE', 
                                  metrics=None, 
                                  decay=None, 
                                  loss_weights=None, 
                                  external_trainable_variables=None)
~~~

~~~python
class deepxde.model.Model.train(iterations=None, 
                                batch_size=None, 
                                display_every=1000, 
                                disregard_previous_best=False, 
                                callbacks=None, 
                                model_restore_path=None, 
                                model_save_path=None, 
                                epochs=None)
~~~

La descripción de los parámetros la pueden encontrar en https://deepxde.readthedocs.io/en/latest/modules/deepxde.html?highlight=deepxde.model#module-deepxde.model. 

In [ ]:
# Modelo
model = dde.Model(data, net)

# Optimizador Adam
model.compile("adam", lr=.001, loss_weights=[0.01, 1, 1], metrics=["l2 relative error"])
losshistory, train_state = model.train(iterations=7000)

## 7. Visualizar la Solución

In [ ]:
dde.saveplot(losshistory, train_state, issave=False, isplot=True) 

## 8. Mejorar la predicción

Vamos a usar un optimizador de segundo orden que ayuda mucho a la convergencia de la solución. Para cambiar los parámetros del optimizador **LBFGS**

~~~
dde.optimizers.set_LBFGS_options
~~~

Estas opciones son

~~~python
deepxde.optimizers.config.set_LBFGS_options(maxcor=100, 
                                            ftol=0, 
                                            gtol=1e-08, 
                                            maxiter=15000, 
                                            maxfun=None, 
                                            maxls=50)
~~~

y se pueden encontrar en https://deepxde.readthedocs.io/en/latest/modules/deepxde.optimizers.html?highlight=optimizers.set_LBFGS_options#deepxde.optimizers.config.set_LBFGS_options

Este optimizador no permite que se hagan tests a medida que se entrena la red.

In [ ]:
# Optimizador L-BFGS-B
dde.optimizers.config.set_LBFGS_options(maxiter=1000)
model.compile("L-BFGS-B")
losshistory, train_state = model.train()

In [ ]:
dde.utils.external.plot_best_state(train_state)

In [ ]:
#dde.utils.external.plot_loss_history(losshistory) #hay un bug y esto no está funcionando